In [3]:
# Importing and installing from our private package
! pip install git+https://github.com/firekind/athena

  Cloning https://github.com/firekind/athena to /tmp/pip-req-build-qrbmstwd
  Running command git clone -q https://github.com/firekind/athena /tmp/pip-req-build-qrbmstwd
     |████████████████████████████████| 71kB 4.7MB/s 
     |████████████████████████████████| 11.6MB 259kB/s 
     |████████████████████████████████| 122kB 59.3MB/s 
     |████████████████████████████████| 49.5MB 55kB/s 
     |████████████████████████████████| 952kB 28.5MB/s 
  Created wheel for athena: filename=athena-1.0.0-cp36-none-any.whl size=38568 sha256=42be8bdd3e6b59dbb89ec7b5de924563700ca4ce108ae1875f79440491881f9a
  Stored in directory: /tmp/pip-ephem-wheel-cache-f23o7sja/wheels/36/d3/7b/d9500a9af2498abc5d48d60c7db37c1456ee745049da352e0b
  Created wheel for albumentations: filename=albumentations-0.4.6-cp36-none-any.whl size=65165 sha256=945531ddc19fce0c3672dedfaf014e99642b37e5b24043de00f8b544c797a979
  Stored in directory: /root/.cache/pip/wheels/c7/f4/89/56d1bee5c421c36c1a951eeb4adcc32fbb82f5344c086efa14
Su

In [4]:
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR
from torchsummary import summary
from athena import Experiment, ClassificationSolver, datasets
from datetime import datetime

In [57]:
print("Current Date/Time: ", datetime.utcnow())

Current Date/Time:  2020-09-26 22:18:37.015882


In [6]:
# Configuring device
device = "cuda" if torch.cuda.is_available() else "cpu"
batch_size = 128 if torch.cuda.is_available() else 64
epochs = 30

In [47]:
class Model(nn.Module):
    def __init__(self):
        dropout_value = 0.1
        super(Model, self).__init__()
        self.x2 = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout2d(dropout_value)
        )
        self.x3 = nn.Sequential(
            nn.Conv2d(19, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Dropout2d(dropout_value)
        )
        self.x4 = nn.MaxPool2d(2, 2)

        self.x5 = nn.Sequential(
            nn.Conv2d(51, 16, kernel_size=1),
            nn.ReLU(),
            nn.BatchNorm2d(16),
        )
        self.x6 = nn.Sequential(
            nn.Conv2d(67, 67, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(67),
            nn.Dropout2d(dropout_value)
        )
        self.x7 = nn.Sequential(
            nn.Conv2d(134, 134, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(134),
            nn.Dropout2d(dropout_value)
        )
        self.x8 = nn.MaxPool2d(2, 2)

        self.x9 = nn.Sequential(
            nn.Conv2d(217, 64, kernel_size=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
        )
        self.x10 = nn.Sequential(
            nn.Conv2d(281, 281, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(281),
            nn.Dropout2d(dropout_value)
        )
        self.x11 = nn.Sequential(
            nn.Conv2d(562, 128, kernel_size=1),
        )

        self.x12 = nn.AdaptiveAvgPool2d((1, 1))
        self.x13 = nn.Conv2d(128, 10, kernel_size = 1)

    def forward(self, x):
        o_x2 = self.x2(x)
        o_x3 = self.x3(torch.cat([x, o_x2], dim=1))
        o_x4 = self.x4(torch.cat([x, o_x2, o_x3], dim=1)) 

        o_x5 = self.x5(o_x4) 
        o_x6 = self.x6(torch.cat([o_x4, o_x5], dim=1))
        o_x7 = self.x7(torch.cat([o_x4, o_x5, o_x6], dim=1)) 
        o_x8 = self.x8(torch.cat([o_x5, o_x6, o_x7], dim=1)) 

        o_x9 = self.x9(o_x8) 
        o_x10 = self.x10(torch.cat([o_x8, o_x9], dim=1)) 
        o_x11 = self.x11(torch.cat([o_x8, o_x9, o_x10], dim=1)) 
        o_x12 = self.x12(o_x11)
        o_x13 = self.x13(o_x12)
        ox_final = o_x13.view(-1, 10)

        return F.log_softmax(ox_final)

In [48]:
summary(Model().to(device), input_size=(3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 32, 32]             448
              ReLU-2           [-1, 16, 32, 32]               0
       BatchNorm2d-3           [-1, 16, 32, 32]              32
         Dropout2d-4           [-1, 16, 32, 32]               0
            Conv2d-5           [-1, 32, 32, 32]           5,504
              ReLU-6           [-1, 32, 32, 32]               0
       BatchNorm2d-7           [-1, 32, 32, 32]              64
         Dropout2d-8           [-1, 32, 32, 32]               0
         MaxPool2d-9           [-1, 51, 16, 16]               0
           Conv2d-10           [-1, 16, 16, 16]             832
             ReLU-11           [-1, 16, 16, 16]               0
      BatchNorm2d-12           [-1, 16, 16, 16]              32
           Conv2d-13           [-1, 67, 16, 16]          40,468
             ReLU-14           [-1, 67,

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:76: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [49]:
exp = (
    Experiment("QuizDNN")
    .model(Model())
    .solver(ClassificationSolver)
        .epochs(40)
        .optimizer(optim.SGD, lr=0.05, momentum=0.9)
        .scheduler(StepLR, step_size=8, gamma=0.1)
        .train_loader(
            datasets.cifar10()
            .batch_size(128)
            .train()
            .use_default_transforms()
            .build()
        )
        .test_loader(
            datasets.cifar10()
            .batch_size(128)
            .train()
            .use_default_transforms()
            .build()
        )
        .device("cuda")
        .build()
    .build()
)

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [50]:
exp.run()

=> Running experiment: QuizDNN
Epoch: 1 / 40
  2/391 [..............................] - ETA: 1:25 - train loss: 2.2091 - train accuracy: 15.1042

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:76: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


391/391 [==============================] - 14s 37ms/step - train loss: 1.6091 - train accuracy: 40.7600
Test set: Average loss: 1.7387, Accuracy: 20400/50000 (40.80%)

Epoch: 2 / 40
391/391 [==============================] - 14s 37ms/step - train loss: 1.3218 - train accuracy: 52.3160
Test set: Average loss: 1.2105, Accuracy: 28424/50000 (56.85%)

Epoch: 3 / 40
391/391 [==============================] - 15s 37ms/step - train loss: 1.1606 - train accuracy: 58.4700
Test set: Average loss: 1.1383, Accuracy: 30449/50000 (60.90%)

Epoch: 4 / 40
391/391 [==============================] - 14s 37ms/step - train loss: 1.0642 - train accuracy: 62.6100
Test set: Average loss: 1.1501, Accuracy: 29742/50000 (59.48%)

Epoch: 5 / 40
391/391 [==============================] - 14s 37ms/step - train loss: 1.0000 - train accuracy: 64.6380
Test set: Average loss: 1.0615, Accuracy: 31676/50000 (63.35%)

Epoch: 6 / 40
391/391 [==============================] - 14s 37ms/step - train loss: 0.9262 - train accu

In [58]:
## Crossed Validation Accuracy of 75% from 9th epoch onwards.